In [ ]:
%cd  ~/enssamble/

In [ ]:
import os, gc
import sys, pdb
import copy, time
import json, random

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path

import matplotlib
import os

import torch

from matplotlib import colors
import matplotlib.pyplot as plt
from colorama import Style, Fore
# %matplotlib inline ipnyb

from dataloader.my_loader import MyDataLoader
from dataloader.OAutoEncoder_loader import ARCDataset

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from visualization.my_vis import ARCPlottor


from models.O_Auto_Encoder import *
from dataloader.my_data_caller import arc_data_collection_figure

datapathType2 = []
arc_data_collection_figure()
with open("/home/hyunseok/enssamble/settings/datapathType2/arc_collections.json",'r') as file:
    path_dict = json.load(file)
    userdata = MyDataLoader("userdata", path_dict)
    userdata.cur_data_mode("train")

# local_path = "/home/hyunseok/enssamble/data/kaggle/"
# with open("/home/hyunseok/enssamble/settings/datapathType1/kaggle_data_file_name.json",'r') as file:
#     path_dict = json.load(file)
#     data = MyDataLoader("arcprize", path_dict, local_path) 
#     data.cur_data_mode("train")

plotter = ARCPlottor()


cmap = colors.ListedColormap(
    ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
     '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])

norm = colors.Normalize(vmin=0, vmax=9)
color_list = ["black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "sky", "brown"]


In [ ]:
!g++ -pthread -lpthread -O3 -std=c++17 -o ./cookedModels/SkleanTree ./models/SkleanTree.cpp
!./cookedModels/SkleanTree

In [ ]:
# BASE_FOLDER = '/home/hyunseok/enssamble/data/kaggle/'
# file_list = [
#     f'{BASE_FOLDER}/arc-agi_training_challenges.json',
#     f'{BASE_FOLDER}/arc-agi_evaluation_challenges.json',
#     f'{BASE_FOLDER}/arc-agi_test_challenges.json'

# ]
# def auto_encoder():
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     cmap = colors.ListedColormap(
#         ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
#         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])

#     norm = colors.Normalize(vmin=0, vmax=10)

#     train_dataset = ARCDataset(file_list)
#     train_dataset.set_dataset()
#     train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

#     Dimension = train_dataset.dim
#     Keys = train_dataset.keys

#     IN_DIM = 100 + 1
#     OUT_DIM = 100
#     LATENT_DIM = 1024
#     model = LSTM(IN_DIM, OUT_DIM, LATENT_DIM).to(device)
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.01)
#     training = Training(model, train_loader, criterion, optimizer, device)
#     training.train()

#     model.load_state_dict(torch.load('model.pth'))

#     sanity = Prediction(model, train_dataset, Dimension, cmap, norm, device)
#     def plot_pic(x):
#         plt.imshow(np.array(x), cmap=cmap, norm=norm)
#         plt.show()
#     for i in range(100):
#         result = sanity.predict(model,[sanity.data.data[i][1],sanity.data.data[i][2]])
#         print(result)
#         output1 = sanity.remove_tail_zeros(sanity.data.data[i][0], torch.round(result * 9), 'out')
#         plot_pic(output1)
# # for i in range(len(pred.data)):
# #     result = pred.predict(model,[pred.data.data[i][1],pred.data.data[i][2]])
# #     output1 = pred.remove_tail_zeros(pred.data.data[i][0], torch.round(result * 9), 'out')
# #     plot_pic(output1)

In [ ]:
from models.symmetry_repariring import *
from models.color_counter import *
from models.different_solvers import *
from models.via_tree import *
from cookedModels.utils import *
from visualization.my_vis import *

from checker.checker import depency_inject_funcs

data_mode = "evaluation"

In [23]:

import json
from datetime import datetime

def make_submission_file(data, solver_name, dataloader, data_mode):
    now = datetime.now()
    metadata = {
        "date": now.strftime("%Y-%m-%d"),  # YYYY-MM-DD 형식
        "time": now.strftime("%H:%M:%S"),   # HH:MM:SS 형식
        "data_mode": data_mode,
        "solver_name": solver_name,
        "data_type": dataloader.module_name
    } 
    json_file_data = {
        'metadata': metadata,
        'submission': data
    }

    # JSON 파일로 저장
    with open('./data/submissions/{}-{}-{}-{}-{}.json'.format(metadata['date'], metadata['time'],metadata['data_type'],metadata['data_mode'],metadata['solver_name'] ), 'w') as json_file:
        json.dump(json_file_data, json_file, indent=4)

In [ ]:
data_mode = "evaluation"
data = userdata
f1 = lambda test_input: predict_repeating(test_input)
f2 = lambda task, test_input: predict_transforms_grid_2x(task, test_input)
f3 = lambda g: predict_chess(grid_filter(g))
f4 = lambda task, test_input: predict_tiles_shape(task, test_input)
f5 = lambda task,test_input: predict_grid_transforms(task, test_input)
f6 = lambda test_input: predict_repeating_mask(test_input)

fdata1 = depency_inject_funcs( data, data_mode, f1, [(check_repeating, [True], {})])
fdata2 = depency_inject_funcs( data, data_mode, f2, [(check_grid, [], {}),(check_sub_grid_2x,[],{})])
fdata3 = depency_inject_funcs( data, data_mode, f3, [(check_grid, [], {}),(check_chess,[ False, True],{})])
fdata4 = depency_inject_funcs( data, data_mode, f4, [(check_tiles_shape, [True], {})],[],{})
fdata5 = depency_inject_funcs( data, data_mode, f5, [(check_grid, [], {}), (check_grid_transforms,[],{})])
fdata6 = depency_inject_funcs( data, data_mode, f6, [(check_sub_mask, [], {})])



In [ ]:
from cookedModels.colors_counter import *
from cookedModels.symmetry_repairing import *
from cookedModels.sklearn_tree import * 

store1 = {}
store2 = {}
store3 = {}
for task_id in data.cur_problem:
    task = data.cur_problem[task_id]
    for i in range(len(task['test'])):
        test_input = np.array(task['test'][i]['input'])
        try:
            prn = []
            ganswer = []
            cooked_symmetry_repairing(store1, task, task_id, i, ganswer, prn, pic_mode=False)
            prn = []
            ganswer = []
            cooked_color_counter(store2, task, task_id, i, ganswer, prn, pic_mode=False)
            prn = []
            ganswer = []
            cooked_sklearn_tree(store3, task, test_input, task_id, prn, pic_mode=False)
        except:
            plotter.check_type(task,"error:"+task_id)
        finally:
            print(task_id)


In [20]:
from cookedModels.arc_dsl_solvers import cooked_arc_dsl_solvers
from dataloader.my_loader import *
from models.arcdsls import solvers

store6 = {}
cooked_arc_dsl_solvers(store6, data, data_mode)

evaluation user data is set
Execution Time: 99.06085443496704 seconds
land_ccw_13_46_87_92_50_02_55 ['solve_ed36ccf7']
landport_rev_01_27_96_35 ['solve_3c9b0459', 'solve_6150a2bd', 'solve_bda2d7a6', 'solve_f76d97a5']
1d_fill_45 ['solve_1f876c06', 'solve_22168020', 'solve_22eb0ac0', 'solve_2c608aff', 'solve_56ff96f3', 'solve_b7249182', 'solve_ba97ae07']
a5f85a15training ['solve_a5f85a15']
port_rev_12_85_37_49_60 ['solve_3c9b0459', 'solve_5521c0d9', 'solve_6150a2bd', 'solve_68b16354', 'solve_bda2d7a6']
74dd1130_syn_3 ['solve_74dd1130', 'solve_9dfd6313']
95990924training ['solve_95990924']
1d_fill_17 ['solve_1f876c06', 'solve_22168020', 'solve_22eb0ac0', 'solve_2c608aff', 'solve_56ff96f3', 'solve_ba97ae07']
landport_rev_72_51_98_46_30 ['solve_3c9b0459', 'solve_6150a2bd', 'solve_bda2d7a6']
landport_ccw_56_31_29_88 ['solve_ed36ccf7']
1d_denoising_mc_2 ['solve_1a07d186', 'solve_3618c87e', 'solve_42a50994', 'solve_952a094c', 'solve_d687bc17']
land_cw_28_71_93_11 ['solve_74dd1130', 'solve_9dfd

{'land_ccw_13_46_87_92_50_02_55': [[[0], [5]]],
 'landport_rev_01_27_96_35': [[[5], [3]], [[5], [3]], [[5], [3]], [[0], [3]]],
 '1d_fill_45': [[[1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    0,
    0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]]],
 'a5f85a15training': [[[0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0],
   [6, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0],
   [0, 4, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0],
   [0, 0, 6, 0, 0, 0, 0, 4, 0, 0, 0, 0],
   [0, 0, 0, 4, 0, 0, 0, 0, 6, 0, 0, 0],
   [0, 0, 0, 0, 6, 0, 0, 0, 0, 4, 0, 0],
   [0, 0, 0, 0, 0, 4, 0, 0, 0

In [24]:
make_submission_file(fdata1, "predict_repeating", data, data_mode)
make_submission_file(fdata2, "predict_transforms_grid_2x", data, data_mode)
make_submission_file(fdata3, "predict_chess", data, data_mode)
make_submission_file(fdata4, "predict_tiles_shape", data, data_mode)
make_submission_file(fdata5, "predict_grid_transforms", data, data_mode)
make_submission_file(fdata6, "predict_repeating_mask", data, data_mode)

make_submission_file(store1, "symmetry_repairing", data, data_mode)
make_submission_file(store2, "colors_counter", data, data_mode)
make_submission_file(store3, "sklearn_tree", data, data_mode)
make_submission_file(store6, "arc_dsl_solvers", data, data_mode)

In [ ]:
def sub_icecube(store):
    with open('./data/submissions/evaluation_sub_icecube.json', 'r') as file:
        icecube = json.load(file)
    store = {}
    for key in icecube:
        print(key,len(icecube[key]))
        for i in range(len(icecube[key])):
            store[key]=[icecube[key][i]['attempt_1'], icecube[key][i]['attempt_2']]
    return store
store7 = {}
store7 = sub_icecube(store7)
make_submission_file(store7, "ice_cuber", data, data_mode)

In [ ]:
import json
import os


def compare_2d_lists(list1, list2):
    return np.array_equal(np.array(list1), np.array(list2))

def check_submissions():
    mem = []
    suc_mem = []
    fail_mem = []
    for dirname, _, filenames in os.walk('/home/hyunseok/enssamble/data/submissions'):
        for filename in filenames:
            suc_dict = {'submission': {}}
            fail_dict = {'submission': {}}

            path = os.path.join(dirname, filename)
            print(path)
            with open(path,'r') as file:
                guess = json.load(file)
                mem.append(guess)

            data_mode='evaluation'
            data.cur_data_mode(data_mode)
            targetgoal = data.cur_problem
            # data.cur_target_goal
            right = 0
            wrong = 0
            if len(guess['submission'].keys()) == 0:
                continue
            for key in guess['submission']:
                
                # plotter.check_type([data.cur_target_goal[key][0], guess['submission'][key][0]],key)
                if(compare_2d_lists(data.cur_problem[key]['test'][0]['output'], guess['submission'][key][0])):
                    right += 1
                    suc_dict['submission'][key] = guess['submission'][key]
                    # print(key, "right")
                else:
                    wrong += 1
                    fail_dict['submission'][key] = guess['submission'][key]
                    # print(key, "wrong")
            suc_dict['metadata'] = guess['metadata']
            fail_dict['metadata'] = guess['metadata']
             
            suc_mem.append(suc_dict)
            fail_mem.append(fail_dict)
            print(right, wrong, len(data.cur_problem[key]['test'][0]['output']), len(guess['submission'][key]))
    return mem, suc_mem, fail_mem


venn, suc_venn, fail_venn = check_submissions()

/home/hyunseok/enssamble/data/submissions/2024-11-05-23:31:54-userdata-evaluation-predict_repeating.json
evaluation user data is set


AttributeError: 'MyDataLoader' object has no attribute 'data'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3

def visualize_set_overlap(dicts):
    # 딕셔너리의 키들을 집합으로 변환
    key_to_extract = "solver_name"
    solvers_name = [ d['solver_name'] for d in dicts] 
    sets = [set(d.keys()) for d in dicts]
    # sets.remove(key_to_extract)
    
    # 두 개의 딕셔너리 비교
    if len(sets) == 2:
        plt.figure(figsize=(8, 8))
        venn = venn2(subsets=sets, set_labels=(solvers_name[0], solvers_name[1]))
        
        # 중복된 키 개수 출력
        print(f"Unique keys in Dict1: {len(sets[0] - sets[1])}")
        print(f"Unique keys in Dict2: {len(sets[1] - sets[0])}")
        print(f"Common keys: {len(sets[0].intersection(sets[1]))}")
        
    # 세 개의 딕셔너리 비교
    elif len(sets) == 3:
        plt.figure(figsize=(8, 8))
        venn = venn3(subsets=sets, set_labels=(solvers_name[0], solvers_name[1], solvers_name[2]))
        
        # 중복된 키 개수 출력
        print(f"Unique keys in Dict1: {len(sets[0] - sets[1] - sets[2])}")
        print(f"Total keys in Dict1: { sets[0] }")
        print(f"Unique keys in Dict2: {len(sets[1] - sets[0] - sets[2])}")
        print(f"Total keys in Dict2: { sets[1] }")
        print(f"Unique keys in Dict3: {len(sets[2] - sets[0] - sets[1])}")
        print(f"Total keys in Dict3: { sets[2] }")

 
        print(f"Common keys among all: {len(sets[0].intersection(sets[1], sets[2]))}")
    
    else:
        print("Error: Please provide 2 or 3 dictionaries only.")
        return
    
    # Venn 다이어그램 출력
    plt.title("Dictionary Key Overlap")
    plt.show()

# 테스트용 딕셔너리
dict1 = {'a': 1, 'b': 2, 'c': 3, 'd': 4}
dict2 = {'b': 2, 'c': 3, 'e': 5, 'f': 6}
dict3 = {'a': 1, 'c': 3, 'g': 7, 'h': 8}

# 시각화 함수 호출


In [ ]:
def find_shared_array_properties(dict1, dict2):
    """
    두 딕셔너리에서 같은 배열을 값으로 가지고 있는 속성의 키 목록을 반환.
    같은 배열이란, 같은 값을 가진 배열이며, 메모리 주소가 다를 수도 있음.
    """
    shared_keys = []

    # 두 딕셔너리의 모든 키에 대해 비교 수행
    for key in dict1:
        if key in dict2:
            # 두 값이 같은 배열인지 비교 (값이 같고, 배열일 경우)
            if isinstance(dict1[key], (list, tuple)) and dict1[key] == dict2[key]:
                shared_keys.append(key)
    
    return shared_keys


In [ ]:
from itertools import combinations

def generate_combinations(items, r):
    """
    주어진 리스트 items에서 r개씩 뽑는 모든 조합을 반환하는 함수.
    """
    return list(combinations(items, r))
# venn, suc_venn, fail_venn
 
# 예제 사용법
items = [ {"solver_name":e['metadata']['solver_name'], **e['submission']} for e in suc_venn]
r = 3
all_combinations = generate_combinations(items, r)

print(f"{len(all_combinations)}개의 조합이 있습니다:")
for combo in  all_combinations:
    visualize_set_overlap(combo)


In [ ]:
# 즉 골라내거나 선발하거나하는 걸 잘함. 즉 임의의 예측치가 있는 오브젝트 선별 과정을 잘함 
data.cur_data_mode(data_mode)
plotter.check_type(data.cur_problem["73ccf9c2"], "73ccf9c2")
plotter.check_type(data.cur_problem['f5aa3634'], 'f5aa3634')
plotter.check_type(data.cur_problem['bf699163'], 'bf699163')


In [ ]:
import itertools
import json

# List of dictionaries to combine
dictionaries = venn
# Extract only submissions for permutation
submissions = [d["submission"] for d in dictionaries]
solver_names = [d["metadata"]["solver_name"] for d in dictionaries]

# Generate all permutations of the submissions and integrate with ensemble sequence
ensemble_results = []
for perm in itertools.permutations(range(len(submissions))):
    combined_submission = {}
    ensemble_sequence = [solver_names[i] for i in perm]
    
    for index in perm:
        for key, value in submissions[index].items():
            combined_submission.setdefault(key, []).append(value)
    
    ensemble_results.append({
        "ensemble_sequence": ensemble_sequence,
        "combined_submission": combined_submission
    })

# Display one of the ensemble results
print(json.dumps(ensemble_results, indent=2))
ensemble_results

In [ ]:
%cd ~/enssamble/


 

In [ ]:
%cd ~/enssamble/
from dataloader.my_data_caller import list_up_arc_data_collections
from dataloader.my_loader import MyDataLoader
import json
with open("/home/hyunseok/enssamble/settings/kaggle_data_file_name.json",'r') as file:
    path_dict = json.load(file)
    
    print(list_up_arc_data_collections())


    data = MyDataLoader("arc-dataset-tama", path_dict) 
data.cur_data_mode("train")

